# Time Series Part I: Estimation, Inference & Forecasting with ARMA

In [ ]:
# load some useful libraries
import statsmodels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter 
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, acf, pacf
import warnings

#### Part 1: Cleaning and Exploring the Covid-19 Data 

In [ ]:
# Load the COVID-19 dataset
df = pd.read_csv('forecasts_and_truth.csv')

display(df.head())

#### a) Load the data, filtering the dataset to focus exclusively on incident cases in the whole of the US. Ensure there are no duplicate rows in the resulting dataset. Plot the time series of cases. 

In [ ]:
df = df[df['target_variable'] == ...]
df = df[df['abbreviation'] == ...]
df = df[['target_end_date','truth_value']]...

display(df.head())

In [ ]:
# Convert the 'target_end_date' column to datetime format
df['target_end_date'] = pd.to_datetime(df['target_end_date'])

# Plot the time series data
...

#### b) Does this data look stationary ? How could we transform the series to "encourage" compatibility with the stable-variance assumption necessary for time-series analysis ? Apply the transformation.

In [ ]:
...

#### c) Augment the series with the first 30 lags of the dependent varible. For each lag, calculate the pearson correlation with the most recent value. Plot this correlation again the order of the lag. 

In [ ]:
from scipy.stats import pearsonr

# Augment the series with the first 30 lags
lags = 30
for lag in range(0, lags + 1):
    df[f'lag_{lag}'] = ... .shift(lag)

# Calculate the Pearson correlation for each lag
correlations = []
for lag in range(0, lags + 1):
    corr = pearsonr(..., df.dropna()[f'lag_{lag}'])[0]
    correlations.append(corr)

# Plotting the correlation against the order of the lag
...

#### d) Interpret the plot. 

...

#### The above plot is called a `correlogram`. It is used to plot the autocorrelation function, which you have just done. The `statmodels` library has its own in-built functions for plotting this, along with another useful plot called the `partial autocorrelation function`. We will get to these in the 2nd time series workshop. 

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Plotting the Autocorrelation Function (ACF)
plt.figure(figsize=(12, 6))
plot_acf(df['log_truth_value'].dropna(), lags=30, alpha=0.05, title='Autocorrelation Function (ACF) for Log-transformed Series')
plt.tight_layout()

# Plotting the Partial Autocorrelation Function (PACF)
plt.figure(figsize=(12, 6))
plot_pacf(df['log_truth_value'].dropna(), lags=30, alpha=0.05, title='Partial Autocorrelation Function (PACF) for Log-transformed Series')
plt.tight_layout()

plt.show()


#### Part II: Random Walks, Forecasts and Inference

#### a) Using a simple random walk, generate a forecast for the 365 days following the last available record in the dataset. Does this generate plausible predictions for the level of new covid cases ?

Note: due to the complicated nature of many-steps-ahead time-series forecasting, which can require stepwise prediction and re-calculation of fitted values, errors etc., we trust the ARIMA library from statmodels to generate multi-step prediction intervals. Note that this does not give us access to simulations for each forecast, and hence we have less flexibility. We will see how we can recover some of this flexibility in the following questions. Note that you `can` still do it the old fashioned way, but this is more straightforward and less messy. 

In [ ]:
# Fit a simple ARIMA model (0,1,0) equivalent to a random walk
model = ARIMA(df['log_truth_value'], order=(0, 1, 0))
model_fit = model.fit()

# Forecast the next 365 days
forecast = model_fit.get_forecast(steps=365)

# Extract the forecast mean and confidence intervals
forecast_mean = forecast.predicted_mean
conf_int = forecast.conf_int(alpha=0.05)  # 95% confidence interval

# Calculate the 70% and 90% confidence intervals
conf_int_70 = forecast.conf_int(alpha=0.30)  # 70% confidence interval
conf_int_90 = forecast.conf_int(alpha=0.10)  # 90% confidence interval

# Convert forecast index to match the dates if necessary
forecast_dates = pd.date_range(start=df['target_end_date'].iloc[-1] + pd.Timedelta(days=1), periods=365)

# Plot the historical data
plt.figure(figsize=(10, 6))
plt.plot(df['target_end_date'], df['log_truth_value'], marker='o', linestyle='-', color='black', markersize=4, label='Historical Data')

# Plot the forecast
plt.plot(forecast_dates, forecast_mean, color='blue', label='Forecast')

# Plot the confidence intervals
plt.fill_between(forecast_dates, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='grey', alpha=1, label='95% Confidence Interval')
plt.fill_between(forecast_dates, conf_int_90.iloc[:, 0], conf_int_90.iloc[:, 1], color='lightcoral', alpha=1, label='90% Confidence Interval')
plt.fill_between(forecast_dates, conf_int_70.iloc[:, 0], conf_int_70.iloc[:, 1], color='red', alpha=1, label='70% Confidence Interval')

# Delineate the start of the forecast with a solid black line
plt.axvline(x=df['target_end_date'].iloc[-1], color='black', linestyle='--')

plt.title('COVID-19 Incident Cases Forecast (log scale)')
plt.xlabel('Date')
plt.ylabel('log(n. cases)')
plt.xticks(rotation=45)
plt.legend(loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

#### b) Now repeat the excercise with a random-walk with drift. 

In [ ]:
# Fit an ARIMA model (0,1,0) with a constant term to model a random walk with drift
model = ARIMA(df['log_truth_value'], order=(0, 1, 0), trend='t')
model_fit = model.fit()

# Forecast the next 365 days
forecast = ...

# Extract the forecast mean and confidence intervals
forecast_mean = ...
conf_int = ...  # 95% confidence interval

# Calculate the 70% and 90% confidence intervals
conf_int_70 = ...  # 70% confidence interval
conf_int_90 = ...  # 90% confidence interval

# Convert forecast index to match the dates if necessary
forecast_dates = ...

# Plot the historical data
plt.figure(figsize=(10, 6))
...

# Plot the forecast
...

# Plot the confidence intervals
...

# Delineate the start of the forecast with a solid black line
...

plt.title('COVID-19 Incident Cases Forecast with Drift (log scale)')
plt.xlabel('Date')
plt.ylabel('log(n. cases)')
plt.xticks(rotation=45)
plt.legend(loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

#### c) From the Random Walk with dirft that you just fit, using the fact that for a univariate random variable we can make inference simply by looking at its marginal distribution, assume a normal distribution and calculate the probability that there will be more than 5 million cases of covid one-year-on from the last observed case in the dataset. 

Hint: This requires you to first estimate the standard deviation of the posterior predictive distribution -- remember the 95% interval is the same as 1.96 standard deviations from the mean on each side for a normal distribution. Then simulate 10000 values from the normal with the ppoint estimate forecast value as the mean, and the estimated standard deviation. Then convert the log-values back to the original scale, and calculate the % of the simulated original-scale values that lie above 5 million. 

In [ ]:
# Get the last row (which corresponds to the forecast interval you're interested in)
...

# Calculate the range by subtracting the lower limit from the upper limit
...

# Estimate the standard deviation by dividing the range by 4
...

In [ ]:
# Define n_simulations
...

# Simulating 10000 future log-values using the normal distribution
simulated_log_values = ...

# Convert the log-values back to the original scale
simulated_values = ...

# Calculate the percentage of simulated values that are above 5 million
probability_above_5_million = ...

probability_above_5_million

#### d) Is the drift parameter statistically significant significant ? Answer this question by plotting the posterior density of the drift parameter. 

In [ ]:
from scipy.stats import multivariate_normal, bernoulli, beta, norm

# Extract the coefficients (betas) and their covariance matrix from the logistic regression fit
beta_mean = ...
beta_cov = ...

# Simulate beta coefficients
simulated_betas = ...

In [ ]:
# Calculate the probability that the distribution is above 0
prob_above_0 = ...

# Plot the distribution of beta
...

#### Part III: Fitting ARIMA models.

#### a) Using the `auto_arima` function, detect the optimal number of AR and MA components necessary to model  the series. 

In [ ]:
from pmdarima import auto_arima

# Detecting the optimal ARIMA parameters
model = auto_arima(..., start_p=0, start_q=0,
                   test='adf',       # Use adf test to find optimal 'd'
                   max_p=5, max_q=5, # Maximum p and q
                   m=1,              # Frequency of the series
                   d=None,           # Let the model determine 'd'
                   seasonal=False,   # No Seasonality
                   start_P=0, 
                   D=0, 
                   trace=True,
                   error_action='ignore',  
                   suppress_warnings=True, 
                   stepwise=True)

print(model.summary())

#### b) Generate forecasts for the next year using this model. Display the 70, 90 and 95% intervals as before. 

In [ ]:
# Extract the selected model order
order = model.order

In [ ]:
# Non-seasonal ARIMA model
model = ARIMA(..., order=order)
model_fit = model.fit()

# Forecast the next 365 days
forecast = model_fit.get_forecast(steps=...)

In [ ]:
# Extract the forecast mean and confidence intervals
forecast_mean = ...
conf_int = ...  # 95% confidence interval

# Calculate the 70% and 90% confidence intervals
conf_int_70 = ...  # 70% confidence interval
conf_int_90 = ...  # 90% confidence interval

# Convert forecast index to match the dates if necessary
forecast_dates = ...

# Plot the historical data
...

# Plot the forecast
...

# Plot the confidence intervals
...

# Delineate the start of the forecast with a solid black line
...

plt.title('COVID-19 Incident Cases Forecast with Drift (log scale)')
plt.xlabel('Date')
plt.ylabel('log(n. cases)')
plt.xticks(rotation=45)
plt.legend(loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

#### b) Now, using a temporally-aware cross-validation, get the point-estimate of the generalisation error of the selected model. Ensure you get the generalisation error on the origincal scale so that it is interpretable by policy makers. 

Note: Why must we make edits to traditional cross-validation ? Because we are not interested in `interpolation` but `extrapolation` -- so we want each fold to have a `future time` it has to forecast, rather than just random time points. 

In [ ]:
# Using a manual function:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np

data = df['log_truth_value']  # Your time series data

n_splits = ... # determine the number of cv splits 
total_size = ... # size of your data (T)
split_size = ...  # Determining roughly equal-sized splits

scores = [] # bucket to put the scores in 

for split in range(1, n_splits + 1):
    
    # Calculate the index to split the data
    split_index = split_size * split
    
    # Ensure we do not go beyond the dataset's size
    # (this can happen due to the `roughly' equallty sized)
    if split_index > total_size:
        break
    
    # Splitting the data into training and test sets
    train, test = data[...], ...(data[...]) # ensure here we are looking at testing on the original scale
    
    # Fit the selected model on the training data
    model = ...
    model_fit = ...
    
    # Forecast the next time steps equivalent to the size of the test set
    forecast = ...(model_fit.forecast(steps=len(test)))
    
    # Calculate and store the RMSE
    mse = ...
    rmse = ... # Calculating RMSE
    scores.append(rmse)

# Calculate average RMSE
average_score = np.mean(scores)
print(f'Average RMSE: {average_score}')


#### c) This cross-validation method above provides a general estimate of the forecast error. But this is not so useful because we know many-steps-ahead forecasts will be worse than few-steps-ahead. Edit the above to use each cross-validation split to estimate up to 30 steps ahead worth of generalisation error. Store this in a new object. Plot this against the steps-ahead to see how the error can be expected to evolve over time. 

Note: there might not be enough observations to check 30 steps ahead in each fold -- check as many as you can ! 

The below I give you `for free`. Play around with changing n_splits, and try to understand the underlying plot. 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import pandas as pd

data = df['log_truth_value']

n_splits = 5  # Define the number of splits for cross-validation
max_forecast_horizon = 30  # Maximum forecast horizon -- 
# note: In the dataset we have weekly readings. Here we are testing the accuracy of the forecasts
# for each of these readings, so the horizon is implicitly at the week-level. 
errors = []  # Using a list to collect arrays of varying lengths

tscv = TimeSeriesSplit(n_splits=n_splits)

for train_index, test_index in tscv.split(data):
    train, test = data.iloc[train_index], np.exp(data.iloc[test_index])
    available_forecast_horizon = min(len(test), max_forecast_horizon)
    
    model = ARIMA(train, order=order)
    model_fit = model.fit()
    
    forecast = np.exp(model_fit.forecast(steps=available_forecast_horizon))
    true_values = test.iloc[:available_forecast_horizon].reset_index(drop=True)
    
    fold_errors = np.zeros(available_forecast_horizon)
    for step in range(available_forecast_horizon):
        # Ensuring correct indexing for both forecast and true_values
        # Convert forecast to a numpy array if it's not already one to standardize indexing
        forecast_values = np.array(forecast) if not isinstance(forecast, np.ndarray) else forecast
        mse = mean_squared_error([true_values.iloc[step]], [forecast_values[step]])
        rmse = np.sqrt(mse)
        fold_errors[step] = rmse
    
    errors.append(fold_errors)

# To handle varying lengths, we need to average errors for each step across all folds differently
max_length = max(len(e) for e in errors)  # Find the maximum forecast horizon across folds
average_errors = np.zeros(max_length)
for i in range(max_length):
    step_errors = [e[i] for e in errors if i < len(e)]  # Collect ith error from each fold if it exists
    average_errors[i] = np.mean(step_errors)  # Average those errors

# Plot the generalization error as a function of forecast horizon
plt.figure(figsize=(10, 6))
plt.plot(range(1, max_length + 1), average_errors, marker='o')
plt.title('Generalization Error up to 30 Steps Ahead')
# Manually format y-axis labels to avoid scientific notation
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: '{:.2f}'.format(x)))
plt.xlabel('Forecast Horizon (Steps Ahead)')
plt.ylabel('RMSE')
plt.grid(True)
plt.show()


There are other ways to conduct this sort of generalisation error estimation -- see here for example: https://openforecast.org/adam/rollingOrigin.html